In [19]:
import os
import pandas as pd
import numpy as np
import csv
from tqdm.notebook import tqdm
import time
from requests_html import HTMLSession
import re
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pyodbc
from tqdm import tqdm
import time

In [20]:
client_id = "d3fcd37a4e40495797a20f338a8558e8"
client_secret = "21d0774b6567456da988d0fbcf0589a3"

In [21]:
auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [22]:
sp = Spotify(auth_manager=SpotifyClientCredentials(client_id="d3fcd37a4e40495797a20f338a8558e8", client_secret="21d0774b6567456da988d0fbcf0589a3"))


In [ ]:
# Hàm crawl thông tin playlist và track từ Spotify, bao gồm cả audio_features
def crawl_playlist(user='spotify', num_playlist=50, replace=True):
    playlist_attributes = [
        'playlist_id',
        'playlist_name',
        'description',
        'num_tracks',
        'num_followers'
    ]

    track_attributes = [
        'track_id',
        'track_name',
        'playlist_id',
        'playlist_name',
        'artist_ids',
        'artist_names',
        'album_id',
        'album_name',
        'track_duration_ms'
    ]
    
    audio_feature_attributes = [
        'track_id',
        'danceability',
        'energy',
        'key_signature',
        'loudness',
        'mode',
        'speechiness',
        'acousticness',
        'instrumentalness',
        'liveness',
        'valence',
        'tempo',
        'time_signature',
        'popularity'
    ]

    # Connect to SQL Server
    try:
        conn = pyodbc.connect('DRIVER={SQL Server};'
                              'SERVER=DESKTOP-KC055B9;'
                              'DATABASE=SPOTIFY_GROUP11;'
                              'Trusted_Connection=yes;')
        cursor = conn.cursor()
        print("Database connected successfully.")
    except Exception as e:
        print(f"Database connection failed: {e}")
        return

    # Tạo bảng nếu chưa tồn tại bảng trong SQL Server
    cursor.execute("""
    IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='Playlists' AND xtype='U')
    CREATE TABLE Playlists (
        playlist_id VARCHAR(255) PRIMARY KEY,
        playlist_name VARCHAR(255),
        description TEXT,
        num_tracks INT,
        num_followers INT
    )
    """)
    cursor.execute("""
    IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='Tracks' AND xtype='U')
    CREATE TABLE Tracks (
        track_id VARCHAR(255) PRIMARY KEY,
        track_name VARCHAR(255),
        playlist_id VARCHAR(255),
        playlist_name VARCHAR(255),
        artist_ids TEXT,
        artist_names TEXT,
        album_id VARCHAR(255),
        album_name VARCHAR(255),
        track_duration_ms INT,
        FOREIGN KEY (playlist_id) REFERENCES Playlists(playlist_id)
    )
    """)
    cursor.execute("""
    IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='Audio_feature' AND xtype='U')
    CREATE TABLE Audio_feature (
        track_id VARCHAR(255) PRIMARY KEY,
        danceability FLOAT,
        energy FLOAT,
        key_signature INT, 
        loudness FLOAT,
        mode INT,
        speechiness FLOAT,
        acousticness FLOAT,
        instrumentalness FLOAT,
        liveness FLOAT,
        valence FLOAT,
        tempo FLOAT,
        time_signature INT,
        popularity INT
    )
    """)
    conn.commit()
    print("Tables created or verified.")

    if replace:
        # Clear existing data if replace is True
        cursor.execute("DELETE FROM Playlists")
        cursor.execute("DELETE FROM Tracks")
        cursor.execute("DELETE FROM Audio_feature")
        conn.commit()
        print("Existing data cleared.")

    # Crawl data
    for i in tqdm(range(0, num_playlist, 50), desc="All playlists"):
        time.sleep(0.1)
        playlists = sp.user_playlists(user, offset=i)

        for pl in tqdm(playlists['items'], desc=f'Playlist {i} to {i+50}'):
            try:
                playlist = sp.playlist(pl['id'])

                # Insert playlist data
                cursor.execute("""
                INSERT INTO Playlists (
                    playlist_id, playlist_name, description, num_tracks, num_followers
                ) VALUES (?, ?, ?, ?, ?)
                """, (
                    playlist['id'],
                    playlist['name'],
                    playlist.get('description', ''),
                    playlist['tracks']['total'],
                    playlist['followers']['total']
                ))
                print(f"Inserted playlist: {playlist['id']} - {playlist['name']}")

                # Insert track data
                for track in playlist['tracks']['items']:
                    track = track['track']
                    cursor.execute("""
                    INSERT INTO Tracks (
                        track_id, track_name, playlist_id, playlist_name,
                        artist_ids, artist_names, album_id, album_name, track_duration_ms
                    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
                    """, (
                        track['id'],
                        track['name'],
                        playlist['id'],
                        playlist['name'],
                        ','.join([artist['id'] for artist in track['artists']]),
                        ','.join([artist['name'] for artist in track['artists']]),
                        track['album']['id'],
                        track['album']['name'],
                        track['duration_ms']
                    ))
                    print(f"Inserted track: {track['id']} - {track['name']}")

                # Insert audio feature data
                audio_features = sp.audio_features([track['id'] for track in playlist['tracks']['items']])
                for features in audio_features:
                    if features:
                        cursor.execute("""
                        INSERT INTO Audio_feature (
                            track_id, danceability, energy, key_signature, loudness, mode,
                            speechiness, acousticness, instrumentalness, liveness,
                            valence, tempo, time_signature, popularity
                        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                        """, (
                            features['id'],
                            features['danceability'],
                            features['energy'],
                            features['key'],
                            features['loudness'],
                            features['mode'],
                            features['speechiness'],
                            features['acousticness'],
                            features['instrumentalness'],
                            features['liveness'],
                            features['valence'],
                            features['tempo'],
                            features['time_signature'],
                            features.get('popularity', 0)
                        ))
                        print(f"Inserted audio features for track: {features['id']}")
            except Exception as e:
                print('Error while processing playlist:', pl['external_urls']['spotify'], e)

    conn.commit()
    print("Data committed to the database.")
    print("Verifying data:")

    # Verify data
    cursor.execute("SELECT COUNT(*) FROM Playlists")
    playlist_count = cursor.fetchone()[0]
    print(f"Total playlists in database: {playlist_count}")

    cursor.execute("SELECT COUNT(*) FROM Tracks")
    track_count = cursor.fetchone()[0]
    print(f"Total tracks in database: {track_count}")

    cursor.execute("SELECT COUNT(*) FROM Audio_feature")
    feature_count = cursor.fetchone()[0]
    print(f"Total audio features in database: {feature_count}")

    conn.close()
    print("Database connection closed.")

crawl_playlist(user="spotify", num_playlist=50, replace=True)


Database connected successfully.
Tables created or verified.
Existing data cleared.


All playlists:   0%|          | 0/1 [00:00<?, ?it/s]

Inserted playlist: 37i9dQZF1DXcBWIGoYBM5M - Today’s Top Hits
Inserted track: 1d7Ptw3qYcfpdLNL5REhtJ - Taste
Inserted track: 2plbrEY59IikOBgBGLjaoe - Die With A Smile
Inserted track: 7ne4VBA60CxGM75vw0EYad - That’s So True
Inserted track: 5vNRhkKd0yEAg8suGBpjeY - APT.
Inserted track: 6dOtVTDdiauQNBQEDOtlAB - BIRDS OF A FEATHER
Inserted track: 0WbMK4wrZ1wFSty9F7FCgu - Good Luck, Babe!
Inserted track: 6MzofobZt2dm0Kf1hTThFz - Diet Pepsi
Inserted track: 1Es7AUAhQvapIcoh3qMKDL - Timeless (with Playboi Carti)
Inserted track: 5N3hjp1WNayUPZrA8kJmJP - Please Please Please
Inserted track: 3WOhcATHxK2SLNeP5W3v1v - Guess featuring billie eilish
Inserted track: 0UYnhUfnUj5adChuAXvLUB - Sailor Song
Inserted track: 3QaPy1KgI7nu9FJEQUgn6h - WILDFLOWER
Inserted track: 2FQrifJ1N335Ljm3TjTVVf - A Bar Song (Tipsy)
Inserted track: 4xdBrk0nFZaP54vvZj0yx7 - HOT TO GO!
Inserted track: 19KlZwqlT3fguP2BeHF1Q1 - Disease
Inserted track: 5G345YEhvleYxQLfYUlEFv - Moonlit Floor (Kiss Me)
Inserted track: 1f18HzFpegq

Inserted playlist: 37i9dQZF1DX0XUsuxWHRQd - RapCaviar
Inserted track: 3tFed7YsjGnIfxeLEQwx3R - Sticky (feat. GloRilla, Sexyy Red & Lil Wayne)
Inserted track: 2C11qhXSCvVzGogwOTP6aZ - SOUTH OF FRANCE - REMIX
Inserted track: 2EKL77lwxvo615Ga7VjEUl - HIM ALL ALONG
Inserted track: 4CkbpelT92r4dVkceDX9g1 - Leave Me Alone
Inserted track: 6afQ4fJDJ9QqU5BKIH99Ej - Insecurities
Error while processing playlist: https://open.spotify.com/playlist/37i9dQZF1DX0XUsuxWHRQd ('23000', "[23000] [Microsoft][ODBC SQL Server Driver][SQL Server]Violation of PRIMARY KEY constraint 'PK__Tracks__24ECC82E34C99E22'. Cannot insert duplicate key in object 'dbo.Tracks'. The duplicate key value is (1Es7AUAhQvapIcoh3qMKDL). (2627) (SQLExecDirectW); [23000] [Microsoft][ODBC SQL Server Driver][SQL Server]The statement has been terminated. (3621)")


Inserted playlist: 37i9dQZF1DX1lVhptIYRda - Hot Country
Inserted track: 54p6vNJwGkt4FTaM8SCsxt - Good News
Error while processing playlist: https://open.spotify.com/playlist/37i9dQZF1DX1lVhptIYRda ('23000', "[23000] [Microsoft][ODBC SQL Server Driver][SQL Server]Violation of PRIMARY KEY constraint 'PK__Tracks__24ECC82E34C99E22'. Cannot insert duplicate key in object 'dbo.Tracks'. The duplicate key value is (7221xIgOnuakPdLqT0F3nP). (2627) (SQLExecDirectW); [23000] [Microsoft][ODBC SQL Server Driver][SQL Server]The statement has been terminated. (3621)")


...

Inserted playlist: 37i9dQZF1DZ06evO4m87u0 - This Is The Black Keys
Inserted track: 5G1sTBGbZT5o4PNRc75RKI - Lonely Boy
Inserted track: 0grFc6klR3hxoHLcgCYsF4 - Howlin' for You
Inserted track: 02bJ6uGeHKfNOhIc9qyA8e - Wild Child
Inserted track: 3Hx7RXqCS7Kzjy2ot2q1Gk - Fever
Inserted track: 5lN1EH25gdiqT1SFALMAq1 - Gold on the Ceiling
Inserted track: 2MVwrvjmcdt4MsYYLCYMt8 - Tighten Up
Inserted track: 5GZq8P4Iy2smHngL07Pcuo - Heavy Soul
Inserted track: 25YlltWXRb9k7KbrEBRuhJ - Lo/Hi
Inserted track: 5lJUXSjRjevVy7I3P87wuK - Your Touch
Inserted track: 1PXsUXSM3LF2XNSkmIldPb - Little Black Submarines
Inserted track: 7ve0nLuGlLfqw0EbL2UZEV - I'll Be Your Man
Inserted track: 23T2c9QnEWy9PvkeDLhHRS - I'm With The Band (feat. Beck)
Inserted track: 7DHlJLCXFlny3WhjnEL9qY - I Forgot To Be Your Lover
Inserted track: 6dU5RxthbuaN31bRbEDlNw - Everlasting Light
Inserted track: 7F24giQdWEKip6mIyGuGs7 - Stay In Your Grave (feat. Alice Cooper)
Inserted track: 3xMQOd1C3TXsjQ3pmzOmkC - Weight of Love
Ins

Inserted playlist: 37i9dQZF1DZ06evO2CLCA8 - This Is Tom Petty and the Heartbreakers
Inserted track: 17S4XrLvF5jlGvGCJHgF51 - Learning To Fly
Inserted track: 3dmqIB2Qxe2XZobw9gXxJ6 - Mary Jane's Last Dance
Inserted track: 53LwBS4bovyljsMlLqGfd6 - American Girl
Inserted track: 4mcZYzoGwPRDdhWVlygiHf - Refugee
Inserted track: 66LhCsc06aTa2Ig7iYPDSP - Stop Draggin' My Heart Around (with Tom Petty and The Heartbreakers) - 2016 Remaster
Inserted track: 63xBnyUVKIupzjHno4wFs3 - Into The Great Wide Open
Inserted track: 1HfB7BLeW5XXIJ9sjCaL5U - Breakdown
Inserted track: 5AzpZ5ADn1AFPxcEBd2Ugf - Don't Do Me Like That
Inserted track: 4ZA0EXmjnZIYguEMf0Mc88 - The Waiting
Inserted track: 5eMZ3EPIMGoWlrrVhxatdv - You Got Lucky
Inserted track: 3BSoskNONnjn05yRO2tYFX - Don't Come Around Here No More
Inserted track: 685sEpLJe80Zzuxz3siOVT - Here Comes My Girl
Inserted track: 2NNpI1bOvnhXcACZ4IdiM3 - I Need To Know
Inserted track: 5SdMXQmfV4OZx5O05986d3 - Even The Losers
Inserted track: 61SdG3F3LncuuMLA

Inserted playlist: 37i9dQZF1DZ06evO3M0Fbi - This Is Nirvana
Inserted track: 4CeeEOM32jQcH3eN9Q2dGj - Smells Like Teen Spirit
Inserted track: 2SHTKB8YYlawTGIuJ2b2ok - About A Girl
Inserted track: 11LmqTE2naFULdEP94AUBa - Heart-Shaped Box
Inserted track: 5gRcv46AMTrosmTOqrOV3Q - Dumb
Inserted track: 2RsAajgo0g7bMCHxwH3Sk0 - Come As You Are
Inserted track: 15VRO9CQwMpbqUYA7e6Hwg - The Man Who Sold The World - Live
Inserted track: 5muVpPu8Fj9fXfDbbqDdrZ - Love Buzz
Inserted track: 1Ic9pKxGSJGM0LKeqf6lGe - All Apologies
Inserted track: 5vHLwhxxlGzmClMcxRRFPr - Lithium
Inserted track: 5Rizj5FLb2AyNKu9esjYY6 - Rape Me
Inserted track: 5O8L4I2S4izFGDjvP7xAKv - You Know You're Right
Inserted track: 4UJmPSJsBsIR1U0N79BU1g - Lake Of Fire - Live
Inserted track: 4gHnSNHs8RyVukKoWdS99f - Something In The Way
Inserted track: 3BtHClmMmURD8UHF2fiyxt - Sliver
Inserted track: 40VSbBSYbPN10vJYeZq4tm - Marigold
Inserted track: 551qy5vUgrUfEUc4dCNfht - Where Did You Sleep Last Night - Live
Inserted track: 30

All playlists: 100%|██████████| 1/1 [00:26<00:00, 26.74s/it]

Inserted playlist: 37i9dQZF1DZ06evO0EhHXi - This Is Ramones
Inserted track: 4KcH1ZRV2W1q7Flq0QqC76 - Blitzkrieg Bop - 2016 Remaster
Inserted track: 5NoQvINZLBV1wMYPdNmReL - I Wanna Be Sedated
Inserted track: 07b5vArZtW08PuEqCw61Ei - Pet Sematary
Inserted track: 61MZN0yyMWSEwrLPUUGfgJ - Sheena Is a Punk Rocker - 2017 Remaster
Inserted track: 29imR7dda5XqwPRFmby7HS - Baby, I Love You
Inserted track: 6yXm3xUj0fcN4dGwBAl9qM - Poison Heart
Inserted track: 0vGCxnTwYZFtOAW3oz28kd - Judy Is a Punk - 2016 Remaster
Inserted track: 2HZGsnMByx3V8KmNW6LqWF - Rockaway Beach - 2017 Remaster
Inserted track: 2j3PFZXmenUxgjjBUIod7F - Bonzo Goes to Bitburg
Inserted track: 4FzPIalyruWIiefI91qe4B - Rock 'n' Roll High School - 2002 Remaster
Inserted track: 55mKuESqpfgqe0kooIkTH2 - Beat on the Brat - 2016 Remaster
Inserted track: 4dyBuBbYZj1iiK0ywYjYzT - California Sun - 2017 Remaster
Inserted track: 53rN8CcgrWP6DZcuQ4Q4og - Needles and Pins - 2018 Remaster
Inserted track: 0N8c4Yf3gp8KnngZggvpqi - The KKK To